# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# MY CODE
# cities Data Cleaning
cities = cities.replace('^\[note \d+\]',np.nan,regex=True)
cities = cities.replace('\[note \d+\]','',regex=True)
cities = cities.replace('[—]+',np.nan,regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]':'Pop'})
cities = cities[['Metropolitan area','Pop','NHL']].dropna()
cities = cities.set_index('NHL')

# NHL Data cleaning
nhl_df = nhl_df.replace('\w+ Division',np.nan,regex=True).dropna()
nhl_df = nhl_df.where(nhl_df['year'] == 2018).dropna()
nhl_df['W'] = pd.to_numeric(nhl_df['W'])
nhl_df['L'] = pd.to_numeric(nhl_df['L'])
nhl_df = nhl_df.replace('\*$','',regex=True)
nhl_df = nhl_df.rename(columns={'team':'NHL'})
nhl_df = nhl_df.set_index('NHL')

# Counting WL ratio
nhl_df['ratio'] = (nhl_df['W']) / (nhl_df['W'] + nhl_df['L'])
nhl_df['ratio'] = pd.to_numeric(nhl_df['ratio'])
# setting average ratio for areas with few teams
nhl_df.loc['New Jersey Devils','ratio'] = (nhl_df.loc['New Jersey Devils','ratio'] +
                                           nhl_df.loc['New York Islanders','ratio'] +
                                           nhl_df.loc['New York Rangers','ratio']) / 3
nhl_df.loc['Los Angeles Kings','ratio'] = (nhl_df.loc['Los Angeles Kings','ratio'] +
                                           nhl_df.loc['Anaheim Ducks','ratio']) / 2

# Renaming teams to fit cities DF
nhl_teams_rename = {
    'Tampa Bay Lightning':'Lightning',
    'Boston Bruins':'Bruins',
    'Toronto Maple Leafs':'Maple Leafs',
    'Florida Panthers':'Panthers',
    'Detroit Red Wings':'Red Wings',
    'Montreal Canadiens':'Canadiens',
    'Ottawa Senators':'Senators',
    'Buffalo Sabres':'Sabres',
    'Washington Capitals':'Capitals',
    'Pittsburgh Penguins':'Penguins',
    'Philadelphia Flyers':'Flyers',
    'Columbus Blue Jackets':'Blue Jackets',
    'New Jersey Devils':'RangersIslandersDevils',
    'Carolina Hurricanes':'Hurricanes',
    'Nashville Predators':'Predators',
    'Winnipeg Jets':'Jets',
    'Minnesota Wild':'Wild',
    'Colorado Avalanche':'Avalanche',
    'St. Louis Blues':'Blues',
    'Dallas Stars':'Stars',
    'Chicago Blackhawks':'Blackhawks',
    'Vegas Golden Knights':'Golden Knights',
    'San Jose Sharks':'Sharks',
    'Los Angeles Kings':'KingsDucks',
    'Calgary Flames':'Flames',
    'Edmonton Oilers':'Oilers',
    'Vancouver Canucks':'Canucks',
    'Arizona Coyotes':'Coyotes'
}

nhl_df = nhl_df.T.rename(columns=nhl_teams_rename).T

# Creating resulting DF
ultimate_nhl_df = pd.merge(cities,nhl_df,how='left',left_index=True,right_index=True)
ultimate_nhl_df['Pop'] = pd.to_numeric(ultimate_nhl_df['Pop'])
ultimate_nhl_df['ratio'] = pd.to_numeric(ultimate_nhl_df['ratio'])

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = ultimate_nhl_df['Pop'] # pass in metropolitan area population from cities
    win_loss_by_region = ultimate_nhl_df['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

print(nhl_correlation())

0.012486162921209912


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# MY CODE
# cities Data Cleaning
cities = cities.replace('^\[note \d+\]',np.nan,regex=True)
cities = cities.replace('\[note \d+\]','',regex=True)
cities = cities.replace('[—]+',np.nan,regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]':'Pop'})
cities = cities[['Metropolitan area','Pop','NBA']].dropna()
cities = cities.set_index('NBA')

# NBA Data cleaning
nba_df = nba_df.where(nba_df['year'] == 2018).dropna()
nba_df['W/L%'] = pd.to_numeric(nba_df['W/L%'])
nba_df = nba_df.replace('\s\(\d+\)$','',regex=True)
nba_df = nba_df.replace('\*$','',regex=True)
nba_df = nba_df.rename(columns={'team':'NBA'})
nba_df = nba_df.set_index('NBA')

# Counting WL ratio
# setting average ratio for areas with few teams
nba_df.loc['New York Knicks','W/L%'] = (nba_df.loc['New York Knicks','W/L%'] +
                                           nba_df.loc['Brooklyn Nets','W/L%']) / 2
nba_df.loc['Los Angeles Clippers','W/L%'] = (nba_df.loc['Los Angeles Clippers','W/L%'] +
                                           nba_df.loc['Los Angeles Lakers','W/L%']) / 2

# Renaming teams to fit cities DF
nba_teams_rename = {
    'Toronto Raptors':'Raptors',
    'Boston Celtics':'Celtics',
    'Philadelphia 76ers':'76ers',
    'Cleveland Cavaliers':'Cavaliers',
    'Indiana Pacers':'Pacers',
    'Miami Heat':'Heat',
    'Milwaukee Bucks':'Bucks',
    'Washington Wizards':'Wizards',
    'Detroit Pistons':'Pistons',
    'Charlotte Hornets':'Hornets',
    'New York Knicks':'KnicksNets',
    'Chicago Bulls':'Bulls',
    'Orlando Magic':'Magic',
    'Atlanta Hawks':'Hawks',
    'Houston Rockets':'Rockets',
    'Golden State Warriors':'Warriors',
    'Portland Trail Blazers':'Trail Blazers',
    'Oklahoma City Thunder':'Thunder',
    'Utah Jazz':'Jazz',
    'New Orleans Pelicans':'Pelicans',
    'San Antonio Spurs':'Spurs',
    'Minnesota Timberwolves':'Timberwolves',
    'Denver Nuggets':'Nuggets',
    'Los Angeles Clippers':'LakersClippers',
    'Sacramento Kings':'Kings',
    'Dallas Mavericks':'Mavericks',
    'Memphis Grizzlies':'Grizzlies',
    'Phoenix Suns':'Suns'
}
nba_df = nba_df.T.rename(columns=nba_teams_rename).T

# Creating resulting DF
ultimate_nba_df = pd.merge(cities,nba_df,how='left',left_index=True,right_index=True)
ultimate_nba_df['Pop'] = pd.to_numeric(ultimate_nba_df['Pop'])
ultimate_nba_df['W/L%'] = pd.to_numeric(ultimate_nba_df['W/L%'])


def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = ultimate_nba_df['Pop'] # pass in metropolitan area population from cities
    win_loss_by_region = ultimate_nba_df['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

print(nba_correlation())

-0.17636350642182938


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# MY CODE
# cities Data Cleaning
cities = cities.replace('^\[note \d+\]',np.nan,regex=True)
cities = cities.replace('\[note \d+\]','',regex=True)
cities = cities.replace('[—]+',np.nan,regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]':'Pop'})
cities = cities[['Metropolitan area','Pop','MLB']].dropna()
cities = cities.set_index('MLB')

# NHL Data cleaning
mlb_df = mlb_df.replace('\w+ Division',np.nan,regex=True).dropna()
mlb_df = mlb_df.where(mlb_df['year'] == 2018).dropna()
mlb_df['W'] = pd.to_numeric(mlb_df['W'])
mlb_df['L'] = pd.to_numeric(mlb_df['L'])
mlb_df = mlb_df.replace('\*$','',regex=True)
mlb_df = mlb_df.rename(columns={'team':'MLB'})
mlb_df = mlb_df.set_index('MLB')

# Counting WL ratio
# setting average ratio for areas with few teams
mlb_df.loc['Chicago White Sox','W-L%'] = (mlb_df.loc['Chicago White Sox','W-L%'] +
                                           mlb_df.loc['Chicago Cubs','W-L%']) / 2
mlb_df.loc['Oakland Athletics','W-L%'] = (mlb_df.loc['Oakland Athletics','W-L%'] +
                                           mlb_df.loc['San Francisco Giants','W-L%']) / 2
mlb_df.loc['Los Angeles Angels','W-L%'] = (mlb_df.loc['Los Angeles Angels','W-L%'] +
                                           mlb_df.loc['Los Angeles Dodgers','W-L%']) / 2
mlb_df.loc['New York Yankees','W-L%'] = (mlb_df.loc['New York Yankees','W-L%'] +
                                           mlb_df.loc['New York Mets','W-L%']) / 2

# Renaming teams to fit cities DF
mlb_teams_rename = {
    'Boston Red Sox':'Red Sox',
    'New York Yankees':'YankeesMets',
    'Tampa Bay Rays':'Rays',
    'Toronto Blue Jays':'Blue Jays',
    'Baltimore Orioles':'Orioles',
    'Cleveland Indians':'Indians',
    'Minnesota Twins':'Twins',
    'Detroit Tigers':'Tigers',
    'Chicago White Sox':'CubsWhite Sox',
    'Kansas City Royals':'Royals',
    'Houston Astros':'Astros',
    'Oakland Athletics':'GiantsAthletics',
    'Seattle Mariners':'Mariners',
    'Los Angeles Angels':'DodgersAngels',
    'Texas Rangers':'Rangers',
    'Atlanta Braves':'Braves',
    'Washington Nationals':'Nationals',
    'Philadelphia Phillies':'Phillies',
    'Miami Marlins':'Marlins',
    'Milwaukee Brewers':'Brewers',
    'St. Louis Cardinals':'Cardinals',
    'Pittsburgh Pirates':'Pirates',
    'Cincinnati Reds':'Reds',
    'Colorado Rockies':'Rockies',
    'Arizona Diamondbacks':'Diamondbacks',
    'San Diego Padres':'Padres'   
}

mlb_df = mlb_df.T.rename(columns=mlb_teams_rename).T

# Creating resulting DF
ultimate_mlb_df = pd.merge(cities,mlb_df,how='left',left_index=True,right_index=True)
ultimate_mlb_df['Pop'] = pd.to_numeric(ultimate_mlb_df['Pop'])
ultimate_mlb_df['W-L%'] = pd.to_numeric(ultimate_mlb_df['W-L%'])


def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = ultimate_mlb_df['Pop'] # pass in metropolitan area population from cities
    win_loss_by_region = ultimate_mlb_df['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

print(mlb_correlation())

0.15003737475409495


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# MY CODE
# cities Data Cleaning
cities = cities.replace('^\[note \d+\]',np.nan,regex=True)
cities = cities.replace('\[note \d+\]','',regex=True)
cities = cities.replace('[—]+',np.nan,regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]':'Pop'})
cities = cities[['Metropolitan area','Pop','NFL']].dropna()
cities = cities.set_index('NFL')

# NHL Data cleaning
nfl_df = nfl_df.replace('\w+ Division',np.nan,regex=True).dropna()
nfl_df = nfl_df.where(nfl_df['year'] == 2018).dropna()
nfl_df = nfl_df.replace('\*$|\+$','',regex=True)
nfl_df = nfl_df.replace('^\wFC',np.nan,regex=True).dropna()
nfl_df = nfl_df.rename(columns={'team':'NFL'})
nfl_df = nfl_df.set_index('NFL')



# Counting WL ratio
nfl_df['W-L%'] = pd.to_numeric(nfl_df['W-L%'])
# setting average ratio for areas with few teams
nfl_df.loc['San Francisco 49ers','W-L%'] = (nfl_df.loc['San Francisco 49ers','W-L%'] +
                                           nfl_df.loc['Oakland Raiders','W-L%']) / 2
nfl_df.loc['Los Angeles Rams','W-L%'] = (nfl_df.loc['Los Angeles Rams','W-L%'] +
                                           nfl_df.loc['Los Angeles Chargers','W-L%']) / 2
nfl_df.loc['New York Jets','W-L%'] = (nfl_df.loc['New York Jets','W-L%'] +
                                           nfl_df.loc['New York Giants','W-L%']) / 2

# Renaming teams to fit cities DF
nfl_teams_rename = {
    'New England Patriots':'Patriots',
    'Miami Dolphins':'Dolphins',
    'Buffalo Bills':'Bills',
    'New York Jets':'GiantsJets',
    'Baltimore Ravens':'Ravens',
    'Pittsburgh Steelers':'Steelers',
    'Cleveland Browns':'Browns',
    'Cincinnati Bengals':'Bengals',
    'Houston Texans':'Texans',
    'Indianapolis Colts':'Colts',
    'Tennessee Titans':'Titans',
    'Jacksonville Jaguars':'Jaguars',
    'Kansas City Chiefs':'Chiefs',
    'Denver Broncos':'Broncos',
    'Dallas Cowboys':'Cowboys',
    'Philadelphia Eagles':'Eagles',
    'Washington Redskins':'Redskins',
    'Chicago Bears':'Bears',
    'Minnesota Vikings':'Vikings',
    'Green Bay Packers':'Packers',
    'Detroit Lions':'Lions',
    'New Orleans Saints':'Saints',
    'Carolina Panthers':'Panthers',
    'Atlanta Falcons':'Falcons',
    'Tampa Bay Buccaneers':'Buccaneers',
    'Los Angeles Rams':'RamsChargers',
    'Seattle Seahawks':'Seahawks',
    'San Francisco 49ers':'49ersRaiders',
    'Arizona Cardinals':'Cardinals'
}

nfl_df = nfl_df.T.rename(columns=nfl_teams_rename).T

# Creating resulting DF
ultimate_nfl_df = pd.merge(cities,nfl_df,how='left',left_index=True,right_index=True)
ultimate_nfl_df['Pop'] = pd.to_numeric(ultimate_nfl_df['Pop'])
ultimate_nfl_df['W-L%'] = pd.to_numeric(ultimate_nfl_df['W-L%'])



def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = ultimate_nfl_df['Pop'] # pass in metropolitan area population from cities
    win_loss_by_region = ultimate_nfl_df['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

print(nfl_correlation())

0.004282141436393033


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# MY CODE
# cities Data Cleaning
cities = cities.replace('^\[note \d+\]',np.nan,regex=True)
cities = cities.replace('\[note \d+\]','',regex=True)
cities = cities.replace('[—]+',np.nan,regex=True)
cities = cities.rename(columns={'Population (2016 est.)[8]':'Pop'})
cities = cities[['Metropolitan area','Pop','NFL','NBA','NHL','MLB']]

# NHL Data cleaning
nhl_df = nhl_df.replace('\w+ Division',np.nan,regex=True).dropna()
nhl_df = nhl_df.where(nhl_df['year'] == 2018).dropna()
nhl_df['W'] = pd.to_numeric(nhl_df['W'])
nhl_df['L'] = pd.to_numeric(nhl_df['L'])
nhl_df = nhl_df.replace('\*$','',regex=True)
nhl_df = nhl_df.rename(columns={'team':'NHL'})
nhl_df = nhl_df.set_index('NHL')

# Counting NHL WL ratio
nhl_df['NHL ratio'] = (nhl_df['W']) / (nhl_df['W'] + nhl_df['L'])
nhl_df['NHL ratio'] = pd.to_numeric(nhl_df['NHL ratio'])
# setting average ratio for areas with few teams
nhl_df.loc['New Jersey Devils','NHL ratio'] = (nhl_df.loc['New Jersey Devils','NHL ratio'] +
                                           nhl_df.loc['New York Islanders','NHL ratio'] +
                                           nhl_df.loc['New York Rangers','NHL ratio']) / 3
nhl_df.loc['Los Angeles Kings','NHL ratio'] = (nhl_df.loc['Los Angeles Kings','NHL ratio'] +
                                           nhl_df.loc['Anaheim Ducks','NHL ratio']) / 2

# Renaming teams to fit cities NHL DF
nhl_teams_rename = {
    'Tampa Bay Lightning':'Lightning',
    'Boston Bruins':'Bruins',
    'Toronto Maple Leafs':'Maple Leafs',
    'Florida Panthers':'Panthers',
    'Detroit Red Wings':'Red Wings',
    'Montreal Canadiens':'Canadiens',
    'Ottawa Senators':'Senators',
    'Buffalo Sabres':'Sabres',
    'Washington Capitals':'Capitals',
    'Pittsburgh Penguins':'Penguins',
    'Philadelphia Flyers':'Flyers',
    'Columbus Blue Jackets':'Blue Jackets',
    'New Jersey Devils':'RangersIslandersDevils',
    'Carolina Hurricanes':'Hurricanes',
    'Nashville Predators':'Predators',
    'Winnipeg Jets':'Jets',
    'Minnesota Wild':'Wild',
    'Colorado Avalanche':'Avalanche',
    'St. Louis Blues':'Blues',
    'Dallas Stars':'Stars',
    'Chicago Blackhawks':'Blackhawks',
    'Vegas Golden Knights':'Golden Knights',
    'San Jose Sharks':'Sharks',
    'Los Angeles Kings':'KingsDucks',
    'Calgary Flames':'Flames',
    'Edmonton Oilers':'Oilers',
    'Vancouver Canucks':'Canucks',
    'Arizona Coyotes':'Coyotes'
}
nhl_df = nhl_df.T.rename(columns=nhl_teams_rename).T
nhl_df = nhl_df.reset_index()


# NBA Data cleaning
nba_df = nba_df.where(nba_df['year'] == 2018).dropna()
nba_df['W/L%'] = pd.to_numeric(nba_df['W/L%'])
nba_df = nba_df.replace('\s\(\d+\)$','',regex=True)
nba_df = nba_df.replace('\*$','',regex=True)
nba_df = nba_df.rename(columns={'team':'NBA','W/L%':'NBA ratio'})
nba_df = nba_df.set_index('NBA')

# Counting NBA WL ratio
# setting average ratio for areas with few teams
nba_df.loc['New York Knicks','NBA ratio'] = (nba_df.loc['New York Knicks','NBA ratio'] +
                                           nba_df.loc['Brooklyn Nets','NBA ratio']) / 2
nba_df.loc['Los Angeles Clippers','NBA ratio'] = (nba_df.loc['Los Angeles Clippers','NBA ratio'] +
                                           nba_df.loc['Los Angeles Lakers','NBA ratio']) / 2

# Renaming teams to fit cities NBA DF
nba_teams_rename = {
    'Toronto Raptors':'Raptors',
    'Boston Celtics':'Celtics',
    'Philadelphia 76ers':'76ers',
    'Cleveland Cavaliers':'Cavaliers',
    'Indiana Pacers':'Pacers',
    'Miami Heat':'Heat',
    'Milwaukee Bucks':'Bucks',
    'Washington Wizards':'Wizards',
    'Detroit Pistons':'Pistons',
    'Charlotte Hornets':'Hornets',
    'New York Knicks':'KnicksNets',
    'Chicago Bulls':'Bulls',
    'Orlando Magic':'Magic',
    'Atlanta Hawks':'Hawks',
    'Houston Rockets':'Rockets',
    'Golden State Warriors':'Warriors',
    'Portland Trail Blazers':'Trail Blazers',
    'Oklahoma City Thunder':'Thunder',
    'Utah Jazz':'Jazz',
    'New Orleans Pelicans':'Pelicans',
    'San Antonio Spurs':'Spurs',
    'Minnesota Timberwolves':'Timberwolves',
    'Denver Nuggets':'Nuggets',
    'Los Angeles Clippers':'LakersClippers',
    'Sacramento Kings':'Kings',
    'Dallas Mavericks':'Mavericks',
    'Memphis Grizzlies':'Grizzlies',
    'Phoenix Suns':'Suns'
}
nba_df = nba_df.T.rename(columns=nba_teams_rename).T
nba_df = nba_df.reset_index()


# MLB Data cleaning
mlb_df = mlb_df.replace('\w+ Division',np.nan,regex=True).dropna()
mlb_df = mlb_df.where(mlb_df['year'] == 2018).dropna()
mlb_df['W'] = pd.to_numeric(mlb_df['W'])
mlb_df['L'] = pd.to_numeric(mlb_df['L'])
mlb_df = mlb_df.replace('\*$','',regex=True)
mlb_df = mlb_df.rename(columns={'team':'MLB','W-L%':'MLB ratio'})
mlb_df = mlb_df.set_index('MLB')

# Counting MLB WL ratio
# setting average ratio for areas with few teams
mlb_df.loc['Chicago White Sox','MLB ratio'] = (mlb_df.loc['Chicago White Sox','MLB ratio'] +
                                           mlb_df.loc['Chicago Cubs','MLB ratio']) / 2
mlb_df.loc['Oakland Athletics','MLB ratio'] = (mlb_df.loc['Oakland Athletics','MLB ratio'] +
                                           mlb_df.loc['San Francisco Giants','MLB ratio']) / 2
mlb_df.loc['Los Angeles Angels','MLB ratio'] = (mlb_df.loc['Los Angeles Angels','MLB ratio'] +
                                           mlb_df.loc['Los Angeles Dodgers','MLB ratio']) / 2
mlb_df.loc['New York Yankees','MLB ratio'] = (mlb_df.loc['New York Yankees','MLB ratio'] +
                                           mlb_df.loc['New York Mets','MLB ratio']) / 2

# Renaming teams to fit cities DF
mlb_teams_rename = {
    'Boston Red Sox':'Red Sox',
    'New York Yankees':'YankeesMets',
    'Tampa Bay Rays':'Rays',
    'Toronto Blue Jays':'Blue Jays',
    'Baltimore Orioles':'Orioles',
    'Cleveland Indians':'Indians',
    'Minnesota Twins':'Twins',
    'Detroit Tigers':'Tigers',
    'Chicago White Sox':'CubsWhite Sox',
    'Kansas City Royals':'Royals',
    'Houston Astros':'Astros',
    'Oakland Athletics':'GiantsAthletics',
    'Seattle Mariners':'Mariners',
    'Los Angeles Angels':'DodgersAngels',
    'Texas Rangers':'Rangers',
    'Atlanta Braves':'Braves',
    'Washington Nationals':'Nationals',
    'Philadelphia Phillies':'Phillies',
    'Miami Marlins':'Marlins',
    'Milwaukee Brewers':'Brewers',
    'St. Louis Cardinals':'Cardinals',
    'Pittsburgh Pirates':'Pirates',
    'Cincinnati Reds':'Reds',
    'Colorado Rockies':'Rockies',
    'Arizona Diamondbacks':'Diamondbacks',
    'San Diego Padres':'Padres'   
}

mlb_df = mlb_df.T.rename(columns=mlb_teams_rename).T
mlb_df = mlb_df.reset_index()



# NFL Data cleaning
nfl_df = nfl_df.replace('\w+ Division',np.nan,regex=True).dropna()
nfl_df = nfl_df.where(nfl_df['year'] == 2018).dropna()
nfl_df = nfl_df.replace('\*$|\+$','',regex=True)
nfl_df = nfl_df.replace('^\wFC',np.nan,regex=True).dropna()
nfl_df = nfl_df.rename(columns={'team':'NFL','W-L%':'NFL ratio'})
nfl_df = nfl_df.set_index('NFL')



# Counting WL ratio
nfl_df['NFL ratio'] = pd.to_numeric(nfl_df['NFL ratio'])
# setting average ratio for areas with few teams
nfl_df.loc['San Francisco 49ers','NFL ratio'] = (nfl_df.loc['San Francisco 49ers','NFL ratio'] +
                                           nfl_df.loc['Oakland Raiders','NFL ratio']) / 2
nfl_df.loc['Los Angeles Rams','NFL ratio'] = (nfl_df.loc['Los Angeles Rams','NFL ratio'] +
                                           nfl_df.loc['Los Angeles Chargers','NFL ratio']) / 2
nfl_df.loc['New York Jets','NFL ratio'] = (nfl_df.loc['New York Jets','NFL ratio'] +
                                           nfl_df.loc['New York Giants','NFL ratio']) / 2

# Renaming teams to fit cities DF
nfl_teams_rename = {
    'New England Patriots':'Patriots',
    'Miami Dolphins':'Dolphins',
    'Buffalo Bills':'Bills',
    'New York Jets':'GiantsJets',
    'Baltimore Ravens':'Ravens',
    'Pittsburgh Steelers':'Steelers',
    'Cleveland Browns':'Browns',
    'Cincinnati Bengals':'Bengals',
    'Houston Texans':'Texans',
    'Indianapolis Colts':'Colts',
    'Tennessee Titans':'Titans',
    'Jacksonville Jaguars':'Jaguars',
    'Kansas City Chiefs':'Chiefs',
    'Denver Broncos':'Broncos',
    'Dallas Cowboys':'Cowboys',
    'Philadelphia Eagles':'Eagles',
    'Washington Redskins':'Redskins',
    'Chicago Bears':'Bears',
    'Minnesota Vikings':'Vikings',
    'Green Bay Packers':'Packers',
    'Detroit Lions':'Lions',
    'New Orleans Saints':'Saints',
    'Carolina Panthers':'Panthers',
    'Atlanta Falcons':'Falcons',
    'Tampa Bay Buccaneers':'Buccaneers',
    'Los Angeles Rams':'RamsChargers',
    'Seattle Seahawks':'Seahawks',
    'San Francisco 49ers':'49ersRaiders',
    'Arizona Cardinals':'Cardinals'
}

nfl_df = nfl_df.T.rename(columns=nfl_teams_rename).T
nfl_df = nfl_df.reset_index()


#Creating ultimate DataFrame
ultimate_df = pd.merge(cities,nhl_df[['NHL','NHL ratio']],how='left',on='NHL',left_index=False,right_index=False)
ultimate_df = pd.merge(ultimate_df,nba_df[['NBA','NBA ratio']],how='left',on='NBA',left_index=False,right_index=False)
ultimate_df = pd.merge(ultimate_df,mlb_df[['MLB','MLB ratio']],how='left',on='MLB',left_index=False,right_index=False)
ultimate_df = pd.merge(ultimate_df,nfl_df[['NFL','NFL ratio']],how='left',on='NFL',left_index=False,right_index=False)



def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for i in sports:
        for j in sports:
            if i != j:
                cols = [i+' ratio',j+' ratio']
                data = ultimate_df[cols].dropna()
                for col in cols:
                    data[col] = pd.to_numeric(data[col])
                p_values.loc[i,j] = stats.ttest_rel(data[i+' ratio'],data[j+' ratio'])[1]
                print('p_value of {} and {} is {}'.format(i,j,p_values.loc[i,j]))
                    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

print(sports_team_performance())

p_value of NFL and NBA is 0.9375088282360184
p_value of NFL and NHL is 0.03031754656891069
p_value of NFL and MLB is 0.8034592132146956
p_value of NBA and NFL is 0.9375088282360184
p_value of NBA and NHL is 0.02238590067909119
p_value of NBA and MLB is 0.9495657419262777
p_value of NHL and NFL is 0.03031754656891069
p_value of NHL and NBA is 0.02238590067909119
p_value of NHL and MLB is 0.0007029593210460182
p_value of MLB and NFL is 0.8034592132146956
p_value of MLB and NBA is 0.9495657419262777
p_value of MLB and NHL is 0.0007029593210460182
          NFL       NBA       NHL       MLB
NFL       NaN  0.937509  0.030318  0.803459
NBA  0.937509       NaN  0.022386  0.949566
NHL  0.030318  0.022386       NaN  0.000703
MLB  0.803459  0.949566  0.000703       NaN
